# Proyecto Cambio de Divisas

In [1]:
!pip install sqlalchemy-redshift

In [2]:
#Cargando los paquetes
#Requests nos proporciona las capacidades para enviar una solicitud HTTP a un servidor.
import requests
import yaml
import json
import pytz
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text
import psycopg2

In [3]:
with open('apikey.yaml', 'r') as file:
    api_keys = yaml.safe_load(file)
api_key = api_keys['api_key']

### Obteniendo el cierre - código que se ejecutará diario

In [4]:
# Extraer la fecha actual para el horario de Mx
timezone = pytz.timezone('America/Mexico_City')
current_time = datetime.now(timezone)
formatted_date = current_time.strftime("%Y-%m-%d")

currencies_list = ['EUR', 'GBP', 'USD', 'PEN', 'BTC', 'KRW', 'INR', 'CNY', 'BRL', 'ARS', 'JPY']
currencies = ','.join(currencies_list)

base_url = f"https://api.apilayer.com/currency_data/timeframe?start_date={formatted_date}&end_date={formatted_date}&source=MXN&currencies={currencies}"

payload = {}
headers= {
  "apikey": api_key
}


response = requests.get(base_url, headers=headers, data=payload)
response.status_code

200

In [5]:
print(json.dumps(response.json(), indent=4))

{
    "success": true,
    "timeframe": true,
    "start_date": "2024-01-16",
    "end_date": "2024-01-16",
    "source": "MXN",
    "quotes": {
        "2024-01-16": {
            "MXNEUR": 0.053405,
            "MXNGBP": 0.04597,
            "MXNUSD": 0.058096,
            "MXNPEN": 0.2151,
            "MXNBTC": 1.345652e-06,
            "MXNKRW": 77.778275,
            "MXNINR": 4.827149,
            "MXNCNY": 0.414021,
            "MXNBRL": 0.286204,
            "MXNARS": 47.531206,
            "MXNJPY": 8.554686
        }
    }
}


In [6]:
datos = json.loads(response.text)

Este dataframe trae únicamente una fila, la cual es el valor actual del día en curso que se estará almacenando en la tabla

In [7]:
df = pd.DataFrame(datos)
df

,success,timeframe,start_date,end_date,source,quotes
2024-01-16,True,True,2024-01-16,2024-01-16,MXN,"{'MXNEUR': 0.053405, 'MXNGBP': 0.04597, 'MXNUS..."


### Limpiando datos

In [8]:
df_quotes = pd.json_normalize(df['quotes'])


# Combinar las nuevas columnas con el DataFrame original
df = pd.concat([df, df_quotes], axis=1).drop('quotes', axis=1)

df

,success,timeframe,start_date,end_date,source,MXNEUR,MXNGBP,MXNUSD,MXNPEN,MXNBTC,MXNKRW,MXNINR,MXNCNY,MXNBRL,MXNARS,MXNJPY
2024-01-16,True,True,2024-01-16,2024-01-16,MXN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,0.053405,0.04597,0.058096,0.2151,0.000001,77.778275,4.827149,0.414021,0.286204,47.531206,8.554686


In [9]:
combined_row = df.iloc[0].combine_first(df.iloc[1])
# Convertir la serie combinada en un DataFrame de una sola fila
df_combined = pd.DataFrame([combined_row])
df_combined.rename(columns=lambda x: x.replace('MXN', '') if 'MXN' in x else x, inplace=True)
df_combined.rename(columns=lambda x: x.replace('start_date', 'date') if 'start_date' in x else x, inplace=True)
columns_to_drop = ['success', 'timeframe','end_date']
df_clean = df_combined.drop(columns=columns_to_drop).reset_index(drop=True)
df_clean

,date,source,EUR,GBP,USD,PEN,BTC,KRW,INR,CNY,BRL,ARS,JPY
0,2024-01-16,MXN,0.053405,0.04597,0.058096,0.2151,0.000001,77.778275,4.827149,0.414021,0.286204,47.531206,8.554686


In [10]:
# Conocer el tipo de dato en el df
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1 non-null      object 
 1   source  1 non-null      object 
 2   EUR     1 non-null      float64
 3   GBP     1 non-null      float64
 4   USD     1 non-null      float64
 5   PEN     1 non-null      float64
 6   BTC     1 non-null      float64
 7   KRW     1 non-null      float64
 8   INR     1 non-null      float64
 9   CNY     1 non-null      float64
 10  BRL     1 non-null      float64
 11  ARS     1 non-null      float64
 12  JPY     1 non-null      float64
dtypes: float64(11), object(2)
memory usage: 232.0+ bytes


### Enviando datos a Redshift

In [11]:
# Cargar credenciales desde el archivo YAML
from sqlalchemy import inspect
with open('credentials.yaml', 'r') as file:
    credenciales = yaml.safe_load(file)['redshift']

# Utilizar credenciales ocultas
username = credenciales['username']
password = credenciales['password']
host = credenciales['host']
port = credenciales['port']
database = credenciales['database']

# Construir la cadena de conexión
cadena_conexion = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Conectar al motor de la base de datos
engine = create_engine(cadena_conexion)
df_clean.to_sql('cambios_divisa', engine, if_exists='append', index=False)


1

### Leer los datos de redshift que fueron cargador previamente (al día actual) + los que ya se habían cargado (histórico)

In [12]:
# Comprobar que los datos fueron cargados exitosamente en redshift al ser llamados
query = "SELECT * FROM cambios_divisa"
existing_data = pd.read_sql(query, engine)
existing_data

,date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy,modified_date
0,2023-02-01,MXN,0.048826,0.043409,0.053770,0.206878,0.000002,65.721149,4.393538,0.362503,0.271805,10.068644,6.916104,2024-01-17 03:27:32.536999
1,2023-02-02,MXN,0.049154,0.043856,0.053601,0.204777,0.000002,65.674506,4.397749,0.360868,0.270722,10.054653,6.901198,2024-01-17 03:27:32.536999
2,2023-02-03,MXN,0.048691,0.043760,0.052721,0.201778,0.000002,65.788509,4.349079,0.357193,0.271648,9.881499,6.916796,2024-01-17 03:27:32.536999
3,2023-02-04,MXN,0.048691,0.043736,0.052721,0.203432,0.000002,65.788509,4.335471,0.357204,0.271611,9.881499,6.916023,2024-01-17 03:27:32.536999
4,2023-02-05,MXN,0.048787,0.043709,0.052644,0.203133,0.000002,65.691804,4.329098,0.356679,0.271211,9.902502,6.949845,2024-01-17 03:27:32.536999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,2024-01-13,MXN,0.054079,0.046512,0.059284,0.219234,0.000001,77.869775,4.912981,0.421543,0.287811,48.308548,8.588138,2024-01-17 03:27:32.536999
347,2024-01-14,MXN,0.054152,0.046538,0.059270,0.219183,0.000001,77.851698,4.911842,0.421445,0.287744,48.388030,8.603362,2024-01-17 03:27:32.536999
348,2024-01-15,MXN,0.054059,0.046540,0.059170,0.218776,0.000001,78.131587,4.902130,0.420523,0.287780,48.382810,8.626216,2024-01-17 03:27:32.536999
349,2024-01-16,MXN,0.053405,0.045970,0.058096,0.215100,0.000001,77.778275,4.827149,0.414021,0.286204,47.531206,8.554686,2024-01-17 03:27:32.536999


### Puede haber duplicados en los datos previos, sin embargo, le pasamos un código para eliminarlos

In [13]:
existing_data['modified_date'] = datetime.now()

# Convertir la columna 'date' a tipo datetime
existing_data['date'] = pd.to_datetime(existing_data['date'])

# Formatear la columna 'date' para mostrar solo la fecha
existing_data['date'] = existing_data['date'].dt.strftime('%Y-%m-%d')

# Ordenar el DataFrame por 'date' de forma descendente y 'modified_date' de forma descendente
existing_data = existing_data.sort_values(by=['date', 'modified_date'], ascending=[False, False])

# Eliminar duplicados basados en la columna 'date' y conservar el primero (el más reciente)
df_combined = existing_data.loc[existing_data.groupby('date')['modified_date'].idxmax()]
df_combined['date'] = pd.to_datetime(df_combined['date'])

df_combined

,date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy,modified_date
0,2023-02-01,MXN,0.048826,0.043409,0.053770,0.206878,0.000002,65.721149,4.393538,0.362503,0.271805,10.068644,6.916104,2024-01-17 03:31:40.522277
1,2023-02-02,MXN,0.049154,0.043856,0.053601,0.204777,0.000002,65.674506,4.397749,0.360868,0.270722,10.054653,6.901198,2024-01-17 03:31:40.522277
2,2023-02-03,MXN,0.048691,0.043760,0.052721,0.201778,0.000002,65.788509,4.349079,0.357193,0.271648,9.881499,6.916796,2024-01-17 03:31:40.522277
3,2023-02-04,MXN,0.048691,0.043736,0.052721,0.203432,0.000002,65.788509,4.335471,0.357204,0.271611,9.881499,6.916023,2024-01-17 03:31:40.522277
4,2023-02-05,MXN,0.048787,0.043709,0.052644,0.203133,0.000002,65.691804,4.329098,0.356679,0.271211,9.902502,6.949845,2024-01-17 03:31:40.522277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2024-01-12,MXN,0.054079,0.046501,0.059284,0.219471,0.000001,77.869775,4.912981,0.421543,0.287775,48.308548,8.588138,2024-01-17 03:31:40.522277
346,2024-01-13,MXN,0.054079,0.046512,0.059284,0.219234,0.000001,77.869775,4.912981,0.421543,0.287811,48.308548,8.588138,2024-01-17 03:31:40.522277
347,2024-01-14,MXN,0.054152,0.046538,0.059270,0.219183,0.000001,77.851698,4.911842,0.421445,0.287744,48.388030,8.603362,2024-01-17 03:31:40.522277
348,2024-01-15,MXN,0.054059,0.046540,0.059170,0.218776,0.000001,78.131587,4.902130,0.420523,0.287780,48.382810,8.626216,2024-01-17 03:31:40.522277


In [14]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 349
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           350 non-null    datetime64[ns]
 1   source         350 non-null    object        
 2   eur            350 non-null    float64       
 3   gbp            350 non-null    float64       
 4   usd            350 non-null    float64       
 5   pen            350 non-null    float64       
 6   btc            350 non-null    float64       
 7   krw            350 non-null    float64       
 8   inr            350 non-null    float64       
 9   cny            350 non-null    float64       
 10  brl            350 non-null    float64       
 11  ars            350 non-null    float64       
 12  jpy            350 non-null    float64       
 13  modified_date  350 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(11), object(1)
memory usage: 41.0+ KB


### Volver a cargar datos pero ahora evitando duplicados

In [15]:
# Escribir los datos combinados y sin duplicados de nuevo en la base de datos
df_combined.to_sql('cambios_divisa', engine, if_exists='replace', index=False)
df_combined

,date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy,modified_date
0,2023-02-01,MXN,0.048826,0.043409,0.053770,0.206878,0.000002,65.721149,4.393538,0.362503,0.271805,10.068644,6.916104,2024-01-17 03:31:40.522277
1,2023-02-02,MXN,0.049154,0.043856,0.053601,0.204777,0.000002,65.674506,4.397749,0.360868,0.270722,10.054653,6.901198,2024-01-17 03:31:40.522277
2,2023-02-03,MXN,0.048691,0.043760,0.052721,0.201778,0.000002,65.788509,4.349079,0.357193,0.271648,9.881499,6.916796,2024-01-17 03:31:40.522277
3,2023-02-04,MXN,0.048691,0.043736,0.052721,0.203432,0.000002,65.788509,4.335471,0.357204,0.271611,9.881499,6.916023,2024-01-17 03:31:40.522277
4,2023-02-05,MXN,0.048787,0.043709,0.052644,0.203133,0.000002,65.691804,4.329098,0.356679,0.271211,9.902502,6.949845,2024-01-17 03:31:40.522277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2024-01-12,MXN,0.054079,0.046501,0.059284,0.219471,0.000001,77.869775,4.912981,0.421543,0.287775,48.308548,8.588138,2024-01-17 03:31:40.522277
346,2024-01-13,MXN,0.054079,0.046512,0.059284,0.219234,0.000001,77.869775,4.912981,0.421543,0.287811,48.308548,8.588138,2024-01-17 03:31:40.522277
347,2024-01-14,MXN,0.054152,0.046538,0.059270,0.219183,0.000001,77.851698,4.911842,0.421445,0.287744,48.388030,8.603362,2024-01-17 03:31:40.522277
348,2024-01-15,MXN,0.054059,0.046540,0.059170,0.218776,0.000001,78.131587,4.902130,0.420523,0.287780,48.382810,8.626216,2024-01-17 03:31:40.522277
